In [83]:
#!pip install yake
#!pip install pdfplumber
#!pip install unstructured
# !pip install pi-heif
# !pip install "unstructured[local-inference,pdf]"
# !pip install langchain-mongodb



In [1]:
import os
os.environ["USER_AGENT"] = "MyLangchainApp/1.0 (saahmathworks@gmail.com)"
from langchain.document_loaders import (
    PyPDFLoader,
    Docx2txtLoader,
    CSVLoader,
    WebBaseLoader,
    DirectoryLoader,
    TextLoader
)
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from langchain_mongodb.vectorstores import MongoDBAtlasVectorSearch
from pymongo import MongoClient
import yake
import datetime
import pymongo
import pytesseract
from pdf2image import convert_from_path
from langchain.schema import Document 
import yake


In [2]:
# Load environment variables
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path='./laws/.env')

# correctly access the variable by its name, which is 'mongo_uri'
MONGO_URI = os.environ.get('mongo_uri')
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# Ensure API keys and URI are set
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found. Please set it in your .env file.")
if not MONGO_URI:
    raise ValueError("MONGO_URI not found. Please set it in your .env file.")

print("Configuration loaded successfully.")

Configuration loaded successfully.


In [3]:
DB_NAME = "legal_db"
COLLECTION_NAME = "legal_documents"

In [4]:
# --- Initialize OpenAI Embeddings ---
# This will be used to generate vectors from your document chunks
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=OPENAI_API_KEY)
print("OpenAI Embeddings model initialized.")

# --- Connect to MongoDB Atlas ---
try:
    client = MongoClient(MONGO_URI)
    db = client[DB_NAME]
    collection = db[COLLECTION_NAME]
    print(f"Connected to MongoDB Atlas database '{DB_NAME}', collection '{COLLECTION_NAME}'.")
    # Ping the database to ensure connection is active
    client.admin.command('ping')
    print("MongoDB connection test successful.")
except Exception as e:
    print(f"Error connecting to MongoDB Atlas: {e}")
    # You might want to halt execution if connection fails
    raise

/tmp/ipykernel_38731/314806001.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=OPENAI_API_KEY)


OpenAI Embeddings model initialized.
Connected to MongoDB Atlas database 'legal_db', collection 'legal_documents'.
MongoDB connection test successful.


In [5]:
# ---------------------------
# 1. Load document with OCR fallback
# ---------------------------

def load_document(file_path, metadata, lang="fra"):
    """
    Load a PDF document. Try PDFPlumber first, and if it fails or returns empty text,
    fall back to OCR using Tesseract.
    Returns a list of LangChain Document objects.
    """
    try:
        loader = PDFPlumberLoader(file_path)
        documents = loader.load()

        # Check if text content is non-empty
        if not any(doc.page_content.strip() for doc in documents):
            raise ValueError("Empty text, switching to OCR...")

        print(f"✅ PDFPlumber loaded {len(documents)} pages successfully.")
        return documents

    except Exception as e:
        print(f"⚠️ PDFPlumber failed ({e}), switching to OCR...")

        # OCR fallback
        try:
            images = convert_from_path(file_path)
            ocr_docs = []
            for i, img in enumerate(images):
                text = pytesseract.image_to_string(img, lang=lang).strip()

                if not text:
                    print(f"⚠️ OCR page {i+1}/{len(images)} is empty, skipping.")
                    continue

                ocr_docs.append(Document(
                    page_content=text,
                    metadata={**metadata, "page": i + 1}
                ))

                print(f"📄 OCR page {i+1}/{len(images)} extracted ({len(text)} chars).")

            print(f"✅ OCR extracted {len(ocr_docs)} pages with text.")
            return ocr_docs

        except Exception as ocr_error:
            print(f"❌ OCR failed: {ocr_error}")
            return []



In [6]:
documents_to_process = [
    {
        "file_path": "./loi-2002-07.pdf",
        "metadata": {
            "titre": "Loi N° 2002-07 portant Code des personnes et de la famille",
            "pays": "Bénin",
            "categorie": "Code des personnes et de la famille",
            "langue": "Français",
            "date_publication": datetime.datetime(2004, 8, 24),
            "url_source": "https://sgg.gouv.bj/doc/loi-2002-07/"
        }
    }
]

In [7]:
docs = []
for doc_info in documents_to_process:
    file_path = doc_info["file_path"]
    metadata = doc_info["metadata"]

    ocr_docs = load_document(file_path, metadata, lang="fra")
    docs.extend(ocr_docs) 


⚠️ PDFPlumber failed (Empty text, switching to OCR...), switching to OCR...
📄 OCR page 1/124 extracted (2148 chars).
📄 OCR page 2/124 extracted (3127 chars).
📄 OCR page 3/124 extracted (2766 chars).
📄 OCR page 4/124 extracted (3226 chars).
📄 OCR page 5/124 extracted (3340 chars).
📄 OCR page 6/124 extracted (3330 chars).
📄 OCR page 7/124 extracted (3321 chars).
📄 OCR page 8/124 extracted (3887 chars).
📄 OCR page 9/124 extracted (3489 chars).
📄 OCR page 10/124 extracted (3590 chars).
📄 OCR page 11/124 extracted (3209 chars).
📄 OCR page 12/124 extracted (2796 chars).
📄 OCR page 13/124 extracted (3377 chars).
📄 OCR page 14/124 extracted (3398 chars).
📄 OCR page 15/124 extracted (3101 chars).
📄 OCR page 16/124 extracted (3853 chars).
📄 OCR page 17/124 extracted (3334 chars).
📄 OCR page 18/124 extracted (3033 chars).
📄 OCR page 19/124 extracted (3087 chars).
📄 OCR page 20/124 extracted (3615 chars).
📄 OCR page 21/124 extracted (3740 chars).
📄 OCR page 22/124 extracted (2758 chars).
📄 OCR pag

In [8]:
print("data type of docs is; ", type(docs), "and it length is: ", len(docs))

data type of docs is;  <class 'list'> and it length is:  124


In [9]:
docs[6]

Document(metadata={'titre': 'Loi N° 2002-07 portant Code des personnes et de la famille', 'pays': 'Bénin', 'categorie': 'Code des personnes et de la famille', 'langue': 'Français', 'date_publication': datetime.datetime(2004, 8, 24, 0, 0), 'url_source': 'https://sgg.gouv.bj/doc/loi-2002-07/', 'page': 7}, page_content="- Les volets n° 2 sont transmis au greffe du tribunal de première instance territorialement\ncompétent ;\n\n- Les volets n° 1 sont remis immédiatement et sans frais au déclarant.\n\nAtticle 39 : Les procurations et autres pièces qui doivent demeurer annexées aux actes de\nl'état civil, sont cotées par référence à l'acte qu'elles concernent, classées chronologiquement par\nnature et date de l'acte et, en fin d'année, enliassées pour être transmises au greffe du tribunal de\npremière instance.\n\nPour chaque registre, l'officier de l'état civil tient en outre, en annexe, un répertoire de\nfeuilles mobiles alphabétiques en double exemplaire, qui sera relié à la fin de chaque 

In [10]:
# ---------------------------
# 2. Split text into chunks
# ---------------------------
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

def split_documents(documents, chunk_size=1000, chunk_overlap=150):
    """
    Split a list of LangChain Document objects into smaller chunks.
    
    Args:
        documents (list[Document]): The documents to split (from load_document).
        chunk_size (int): Maximum size of each text chunk in characters.
        chunk_overlap (int): Number of characters to overlap between chunks.

    Returns:
        list[Document]: A new list of Document objects, where each page is split
                        into smaller chunks with preserved metadata.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )

    chunks = text_splitter.split_documents(documents)
    print(f"✅ Split {len(documents)} documents into {len(chunks)} chunks.")
    return chunks


In [11]:
# Apply the splitter
chunks = split_documents(docs, chunk_size=1000, chunk_overlap=150)




✅ Split 124 documents into 494 chunks.


In [15]:
# Quick check: print first 300 chars of the first 2 chunks
for i, chunk in enumerate(chunks[15:20]):
    print(f"\n--- Chunk {i+1} (page {chunk.metadata.get('page', 'N/A')}) ---")
    print(f"\n--- Chunk {i+1} (page {chunk.metadata}) ---")
    print(chunk.page_content)
    # print(chunk.page_content[:1000], "...")


--- Chunk 1 (page 5) ---

--- Chunk 1 (page {'titre': 'Loi N° 2002-07 portant Code des personnes et de la famille', 'pays': 'Bénin', 'categorie': 'Code des personnes et de la famille', 'langue': 'Français', 'date_publication': datetime.datetime(2004, 8, 24, 0, 0), 'url_source': 'https://sgg.gouv.bj/doc/loi-2002-07/', 'page': 5}) ---
Article 28 : L'affaire est instruite et jugée en chambre du conseil. Tous les actes de la
procédure ainsi que les expéditions et extraits desdits actes sont dispensés du timbre et enregistrés
gratis. Si le tribunal estime que le décès n'est pas suffisamment établi, il peut ordonner toute mesure
d'information complémentaire et requérir notamment une enquête administrative sur les circonstances
de la disparition. Si le décès est déclaré, sa date doit être fixée en tenant compte des présomptions
tirées des circonstances de la cause et, à défaut, au jour de la disparition. Cette date ne doit jamais être
indéterminée.

Article 29 : Le dispositif du jugement déc

In [16]:
print("data type of chunks is; ", type(chunks), "and it length is: ", len(chunks))

data type of chunks is;  <class 'list'> and it length is:  494


In [23]:
import yake
import re
from typing import List, Dict, Any
from langchain.docstore.document import Document
import pymongo

# ---------------------------
# Enhanced Pre-processing with Article Reference Extraction
# ---------------------------

def extract_article_references(text: str) -> List[str]:
    """
    Extract all article references from text with improved pattern matching.
    """
    # Enhanced patterns for legal document references
    patterns = [
        r"article\s+(\d+)(?:\s+de\s+la\s+même\s+loi)?",  # "article 7" or "article 7 de la même loi"
        r"art\.?\s*(\d+)",  # "art. 7" or "art 7"
        r"article\s+(\d+)\s+ci-?dessus",  # "article 7 ci-dessus"
        r"article\s+(\d+)\s+ci-?dessous",  # "article 7 ci-dessous"
        r"article\s+(\d+)\s+précédent",  # "article 7 précédent"
        r"article\s+(\d+)\s+suivant",  # "article 7 suivant"
        r"aux\s+articles?\s+(\d+)",  # "aux articles 7 et 8"
        r"l'article\s+(\d+)",  # "l'article 7"
        r"des?\s+articles?\s+(\d+)",  # "de l'article 7"
    ]
    
    references = set()
    
    for pattern in patterns:
        matches = re.findall(pattern, text, re.IGNORECASE | re.UNICODE)
        for match in matches:
            # Handle multiple articles in one reference: "articles 7, 8 et 9"
            if ',' in match or 'et' in match:
                # Split complex references
                numbers = re.findall(r'\d+', match)
                references.update(numbers)
            else:
                references.add(match)
    
    return list(references)

In [24]:
# Test function
content ="""
Article 28 : L'affaire est instruite et jugée en chambre du conseil. Tous les actes de la
procédure ainsi que les expéditions et extraits desdits actes sont dispensés du timbre et enregistrés
gratis. Si le tribunal estime que le décès n'est pas suffisamment établi, il peut ordonner toute mesure
d'information complémentaire et requérir notamment une enquête administrative sur les circonstances
de la disparition. Si le décès est déclaré, sa date doit être fixée en tenant compte des présomptions
tirées des circonstances de la cause et, à défaut, au jour de la disparition. Cette date ne doit jamais être
indéterminée.

Article 29 : Le dispositif du jugement déclaratif de décès doit être transcrit, selon les
modalités prévues à l'article 84, sur les registres de l'état civil du lieu réel ou présumé du décès et, le
cas échéant, sur ceux du lieu du dernier domicile.
"""
extract_article_references(content)

['28', '84', '29']

In [25]:
def extract_article_numbers_from_chunk(text: str) -> List[str]:
    """
    Extract ALL article numbers defined in a chunk.
    """
    # Pattern to match article definitions
    patterns = [
        r"Article\s+(\d+)\s*[:.-]",  # "Article 28 :"
        r"ARTICLE\s+(\d+)\s*[:.-]",  # "ARTICLE 28 :"
        r"Art\.?\s*(\d+)\s*[:.-]",   # "Art. 28 :"
    ]
    
    articles = set()
    
    for pattern in patterns:
        matches = re.findall(pattern, text)
        articles.update(matches)
    
    return list(articles)

In [26]:
extract_article_numbers_from_chunk(content)

['28', '29']

In [28]:
extract_articles_from_query("Que dit l'article 14 du code penal")

['14']

In [30]:
def embed_chunks_article_focused(chunks, embedding_model, lang="fr"):
    """
    Embedding function focused on article extraction.
    """
    embedded_chunks = []

    for chunk in chunks:
        content = chunk.page_content.strip()
        if not content:
            continue

        # Extract article references and articles defined in this chunk
        article_references = extract_article_references(content)
        articles_in_chunk = extract_article_numbers_from_chunk(content)

        try:
            embedding_vector = embedding_model.embed_query(content)
        except Exception as e:
            print(f"❌ Error creating embedding: {e}. Skipping chunk.")
            continue

        embedded_chunks.append({
            "contenu": content,
            "vecteur_embedding": embedding_vector,
            "article_references": article_references,
            "articles_in_chunk": articles_in_chunk,
            "metadata": chunk.metadata  # Keep original metadata
        })

    print(f"✅ Generated embeddings for {len(embedded_chunks)} chunks")
    
    # Log article statistics
    total_articles = sum(len(chunk["articles_in_chunk"]) for chunk in embedded_chunks)
    total_references = sum(len(chunk["article_references"]) for chunk in embedded_chunks)
    print(f"📊 Found {total_articles} article definitions and {total_references} article references")
    
    return embedded_chunks

In [31]:
# Step 3: Embed chunks
embedded_chunks = embed_chunks_article_focused(chunks, embedding_model)

✅ Generated embeddings for 494 chunks with article reference analysis.
📊 Found 817 article definitions and 931 article references across all chunks.


In [32]:
type(embedded_chunks)

list

In [33]:
type(embedded_chunks[5])

dict

In [37]:
#embedded_chunks[5]

In [38]:
def temporarily_disable_validation():
    """
    Temporarily disable schema validation for updates.
    """
    try:
        client = MongoClient(MONGO_URI)
        db = client[DB_NAME]
        
        # Disable validation
        db.command("collMod", COLLECTION_NAME, validator={})
        print("🔓 Schema validation temporarily disabled")
        
        # Process your documents here...
        
        # Re-enable validation
        # db.command("collMod", COLLECTION_NAME, validator=schema_validator)
        # print("🔒 Schema validation re-enabled")
        
    except Exception as e:
        print(f"Error: {e}")
    finally:
        client.close()

In [42]:
temporarily_disable_validation()

In [36]:

# ---------------------------
# 4. Save to MongoDB
# ---------------------------
def save_to_mongodb_article_focused(embedded_chunks, collection, base_metadata, batch_size=100):
    """
    Save chunks with article reference resolution (simplified, no keywords).
    """
    if not embedded_chunks:
        print("⚠️ No chunks to save.")
        return

    # Delete existing documents with same url_source
    existing_count = collection.count_documents({"url_source": base_metadata['url_source']})
    if existing_count > 0:
        result = collection.delete_many({"url_source": base_metadata['url_source']})
        print(f"🗑️ Deleted {result.deleted_count} existing documents")

    # Build article mapping for the entire document
    article_mapping = {}
    for chunk in embedded_chunks:
        for article_num in chunk["articles_in_chunk"]:
            article_mapping[article_num] = chunk["contenu"]

    print(f"📚 Built article mapping with {len(article_mapping)} articles")

    # Prepare documents for insertion
    documents_to_insert = []
    for chunk in embedded_chunks:
        # Resolve references within the same document
        resolved_references = {}
        for ref_article in chunk["article_references"]:
            if ref_article in article_mapping:
                resolved_references[ref_article] = article_mapping[ref_article]

        doc_to_save = {
            **base_metadata,
            "contenu": chunk["contenu"],
            "vecteur_embedding": chunk["vecteur_embedding"],
            "article_references": chunk["article_references"],
            "articles_in_chunk": chunk["articles_in_chunk"],
            "resolved_references": resolved_references,
            "page": chunk["metadata"].get("page", "N/A")
        }

        documents_to_insert.append(doc_to_save)

    # Insert in batches
    try:
        total_inserted = 0
        for i in range(0, len(documents_to_insert), batch_size):
            batch = documents_to_insert[i:i + batch_size]
            result = collection.insert_many(batch)
            inserted_count = len(result.inserted_ids)
            total_inserted += inserted_count
            
            print(f"✅ Inserted batch {i//batch_size + 1} ({inserted_count} docs)")
            
        print(f"🎉 Successfully stored {total_inserted} chunks with article references")
            
    except Exception as e:
        print(f"❌ Failed to insert into MongoDB: {e}")

In [41]:
metadata

{'titre': 'Loi N° 2002-07 portant Code des personnes et de la famille',
 'pays': 'Bénin',
 'categorie': 'Code des personnes et de la famille',
 'langue': 'Français',
 'date_publication': datetime.datetime(2004, 8, 24, 0, 0),
 'url_source': 'https://sgg.gouv.bj/doc/loi-2002-07/'}

In [44]:
embedded_chunks[5].get('metadata')

{'titre': 'Loi N° 2002-07 portant Code des personnes et de la famille',
 'pays': 'Bénin',
 'categorie': 'Code des personnes et de la famille',
 'langue': 'Français',
 'date_publication': datetime.datetime(2004, 8, 24, 0, 0),
 'url_source': 'https://sgg.gouv.bj/doc/loi-2002-07/',
 'page': 2}

In [40]:
# Step 4: Save to MongoDB

save_to_mongodb_article_focused(embedded_chunks, collection, metadata)

🗑️ Deleted 494 existing documents
📚 Built article mapping with 781 articles
✅ Inserted batch 1 (100 docs)
✅ Inserted batch 2 (100 docs)
✅ Inserted batch 3 (100 docs)
✅ Inserted batch 4 (100 docs)
✅ Inserted batch 5 (94 docs)
🎉 Successfully stored 494 chunks with article references


In [ ]:
# test 

In [45]:
def enhance_with_article_context(results, collection):
    """
    Enhance search results with referenced article content.
    """
    enhanced_results = []
    
    for result in results:
        enhanced_results.append(result)
        
        # Extract article references from metadata
        article_refs = result.metadata.get('article_references', [])
        resolved_refs = result.metadata.get('resolved_references', {})
        
        # Add referenced articles as additional context
        for article_num in article_refs[:3]:  # Limit to top 3 references
            if article_num in resolved_refs:
                # Create a pseudo-document for the reference
                from langchain.schema import Document
                ref_doc = Document(
                    page_content=f"Article {article_num} (Référencé): {resolved_refs[article_num][:500]}...",
                    metadata={
                        **result.metadata,
                        "is_reference": True,
                        "referenced_article": article_num,
                        "original_article": result.metadata.get('articles_in_chunk', [])
                    }
                )
                enhanced_results.append(ref_doc)
    
    return enhanced_results

In [46]:
# 1. Create basic retriever
vectorstore = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embedding_model,
    index_name="vector_index",
    text_key="contenu",
    embedding_key="vecteur_embedding"
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [54]:
import re

def smart_legal_query(user_query):
    """
    Fully automatic detection of articles, country, law, and category from natural language.
    """
    
    pre_filter = {}
    detected_articles = []
    
    query_lower = user_query.lower()
    
    # 1. AUTO-DETECT ARTICLE NUMBERS
    article_patterns = [
        r'article[s]?\s+(\d+(?:\s+(?:et|à|\-)\s+\d+)*)',  # article 125, articles 145 et 150
        r'art\.?\s*(\d+(?:\s+(?:et|à|\-)\s+\d+)*)',       # art. 125, art 130-135
        r'articles?\s+(\d+)\s*à\s*(\d+)',                 # article 125 à 130
        r'art\.?\s*(\d+)\s*au\s*(\d+)',                   # art. 125 au 130
    ]
    
    for pattern in article_patterns:
        matches = re.findall(pattern, query_lower)
        for match in matches:
            if isinstance(match, tuple):
                # Handle ranges like "125 à 130"
                numbers = []
                for num in match:
                    if num.isdigit():
                        numbers.append(num)
                detected_articles.extend(numbers)
            else:
                # Handle single articles or lists
                numbers = re.findall(r'\d+', match)
                detected_articles.extend(numbers)
    
    # Remove duplicates and sort
    detected_articles = sorted(list(set(detected_articles)))
    
    # 2. AUTO-DETECT COUNTRY (your existing code)
    country_keywords = {
        "bénin": "Bénin", "béninoise": "Bénin", "béninois": "Bénin",
        "madagascar": "Madagascar", "malgache": "Madagascar",
        # Add more...
    }
    
    for keyword, country in country_keywords.items():
        if keyword in query_lower:
            pre_filter["pays"] = country
            break
    
    # 3. AUTO-DETECT LAW TITLE
    law_keywords = {
        "code des personnes": "Loi N° 2002-07 portant Code des personnes et de la famille",
        "code famille": "Loi N° 2002-07 portant Code des personnes et de la famille", 
        "droit familial": "Loi N° 2002-07 portant Code des personnes et de la famille",
        "code civil": "Code Civil",  # Example for other laws
        # Add more...
    }
    
    for keyword, law_title in law_keywords.items():
        if keyword in query_lower:
            pre_filter["titre"] = law_title
            break
    
    # 4. AUTO-DETECT CATEGORY
    category_keywords = {
        "mariage": "Code des personnes et de la famille",
        "divorce": "Code des personnes et de la famille",
        "héritage": "Code des personnes et de la famille", 
        "succession": "Code des personnes et de la famille",
        "adoption": "Code des personnes et de la famille",
        # Add more...
    }
    
    for keyword, category in category_keywords.items():
        if keyword in query_lower:
            pre_filter["categorie"] = category
            break
    
    # 5. ENHANCE QUERY BASED ON DETECTED ARTICLES
    if detected_articles:
        query = f"article {' '.join(detected_articles)} {user_query}"
        print(f"🔢 Auto-detected articles: {detected_articles}")
    else:
        query = user_query
    
    # 6. PERFORM SEARCH
    if pre_filter:
        print(f"🌍 Auto-detected filters: {pre_filter}")
        docs = vectorstore.similarity_search(query, k=10, pre_filter=pre_filter)
    else:
        docs = vectorstore.similarity_search(query, k=10)
    
    # 7. ENHANCE WITH REFERENCES
    enhanced_docs = enhance_with_article_context(docs, collection)
    
    return enhanced_docs, detected_articles, pre_filter

In [55]:
response1 = smart_legal_query("Quelles sont les conditions de mariage au Bénin?")

🌍 Auto-detected filters: {'pays': 'Bénin', 'categorie': 'Code des personnes et de la famille'}


In [56]:
# Natural language queries - no parameters needed!
results, articles, filters = smart_legal_query("Que dit l'article 125 sur le mariage au Bénin?")

🔢 Auto-detected articles: ['125']
🌍 Auto-detected filters: {'pays': 'Bénin', 'categorie': 'Code des personnes et de la famille'}


In [60]:
results, articles, filters = smart_legal_query("Expliquez les articles 130 et 145 concernant le divorce")

🔢 Auto-detected articles: ['130', '145']
🌍 Auto-detected filters: {'categorie': 'Code des personnes et de la famille'}


In [61]:
results

[Document(id='68d3d5d9d2c28dba269c7092', metadata={'_id': '68d3d5d9d2c28dba269c7092', 'titre': 'Loi N° 2002-07 portant Code des personnes et de la famille', 'pays': 'Bénin', 'categorie': 'Code des personnes et de la famille', 'langue': 'Français', 'date_publication': '2004-08-24T00:00:00', 'url_source': 'https://sgg.gouv.bj/doc/loi-2002-07/', 'article_references': ['259', '260', '255', '258', '261'], 'articles_in_chunk': ['258', '259', '260', '261'], 'resolved_references': {'259': "Article 258 : Le dispositif du jugement ou de l'arrêt qui prononce le divorce doit énoncer, le\ncas échéant, la date de la décision ayant autorisé les époux à résider séparément. Cette date doit\nfigurer dans la mention ou dans la transcription faite en application de l'article 255.\n\nArticle 259 : En cas de divorce par consentement mutuel, il est fait masse des dépens qui\nsont mis pour moitié à la charge de chaque partie.\n\nArticle 260 : Il est fait de même masse et partage des dépens en cas de rejet de 

In [47]:
doc = collection.find_one()
print("Content:", doc.get("contenu"))
print("Vector length:", len(doc.get("vecteur_embedding", [])))


Content: FE.-
REPUBLIQUE DU BENIN

LOI N° 2002-07 DU 24 AOUT 2004

portant Code des personnes et de la famille.

-  L’ASSEMBLEE NATIONALE à délibéré et adopté en sa séance du 07 juin 2002, puis
en sa séance du 14 juin 2004, suite à la décision DCC 02-144 du 23 décembre 2002 de la
Cour Constitutionnelle Pour mise en conformité à la Constitution ;

- Suite à la Décision de conformité à la Constitution DCC 04-083 du 20 août 2004 de ja
Cour Constitutionnelle ;

- LE PRESIDENT DE LA REPUBLIQUE promulgue la loi dont la teneur suit :

LIVRE PREMIER : DES PERSONNES

TITRE PREMIER : DES PERSONNES PHYSIQUES ET MORALES

CHAPITRE 1 : DES DISPOSITIONS GENERALES

Article 1”  : Toute personne humaine, sans distinction aucune notamment de race, de couleur,
de sexe, de réligion, de langue, d’opinion politique ou de toute autre opinion , d’origine nationale
où sociale, de fortune, de naissance ou de toute autre situation, est sujet de droit, de sa naissance
à son décès.
Vector length: 1536


In [62]:
query = "La décision qui autorise le changement de nom profite au requérant et à ses enfants"

results, articles, filters = smart_legal_query(query)


# --- 3. Inspect results ---
for i, doc in enumerate(results, start=1):
    print(f"\n📄 Result {i}")
    print("Titre:", doc.metadata.get("titre"))
    print("Pays:", doc.metadata.get("pays"))
    print("Catégorie:", doc.metadata.get("categorie"))
    print("Contenu (preview):", doc.page_content[:200], "…")



📄 Result 1
Titre: Loi N° 2002-07 portant Code des personnes et de la famille
Pays: Bénin
Catégorie: Code des personnes et de la famille
Contenu (preview): L'adjonction ou la radiation de prénoms peut être autorisée dans les mêmes conditions.

@ La requête est présentée au tribunal dans le ressort duquel le requérant est né, et au
tribunal de première in …

📄 Result 2
Titre: Loi N° 2002-07 portant Code des personnes et de la famille
Pays: Bénin
Catégorie: Code des personnes et de la famille
Contenu (preview): expédition du jugement ou de l'arrêt, accompagnée d'un certificat délivré par le greffier et duquel il
résulte que le jugement ou l'arrêt est devenu définitif.

Article 11 : Il peut être procédé à des …

📄 Result 3
Titre: Loi N° 2002-07 portant Code des personnes et de la famille
Pays: Bénin
Catégorie: Code des personnes et de la famille
Contenu (preview): Article 13 (Référencé): expédition du jugement ou de l'arrêt, accompagnée d'un certificat délivré par le greffier et duquel il

In [27]:
def extract_articles_from_query(query: str) -> List[str]:
    """
    Extract article numbers specifically from user queries.
    More permissive patterns for natural language queries.
    """
    patterns = [
        r"article\s+(\d+)",
        r"art\.?\s*(\d+)",
        r"l'article\s+(\d+)",
        r"article\s+n°?\s*(\d+)",
        r"art\s+(\d+)",
    ]
    
    articles = set()
    
    for pattern in patterns:
        matches = re.findall(pattern, query, re.IGNORECASE)
        articles.update(matches)
    
    return list(articles)

# RAG App implementation

Next step